# Tạo model dự đoán

In [ ]:
import pandas as pd

from loaddata import data_engineered as data

In [ ]:
data.head(5)

Chúng ta đã chứng minh được là những feature có hiện có có quan hệ với việc sống/chết của 1 cá nhân trên tàu Titanic. Công đoạn cuối cùng là code và train 1 Machine Learning model để dự đoán 1 người sẽ sống hay chết nếu ở trên tàu Titanic.

## Ngoài lề: Machine Learning (ML)

Định nghĩa: là ngành khoa học cho phép máy tính "học" trực tiếp từ dữ liệu thay vì được lập trình cụ thể ([Wikipedia](https://en.wikipedia.org/wiki/Machine_learning))

Tức là thay vì
```python
survival_prob = 0.5
if data['Sex'] == 'male':
    survival_prob -= 0.1  # Mấy con số này con người phải tìm, có khi cả đời không ra
if data['Pclass'] == 1:
    survival_prob += 0.25
...
prediction = 'Survive' if survival_prob > 0.5 else 'Die'
```
chỉ cần:
```python
labels = data['Survived']
features = data.drop(columns=['Survived'])
model = MachineLearningModel(some_hyper_parameters)
model.learn(features, labels)
predictions = model.predict(new_data)
# Con người không cần tìm ra cái gì cả, máy tự tìm
```

Có 3 nhánh chính trong ML:
- Supervised learning: học với data có label, mục đích là sau khi học thì, model nhận được input mới dự đoán được output tương ứng
- Unsupervised learning: học với data không có label, mục đích là nhóm các dữ liệu tương tự vào cùng nhóm
- Reinforcement learning: máy thực hiện hành động và nhận được điểm số cho hành động đó, từ đó tìm cách giải quyết bài toán với điểm số tối đa

Hiện tại dạng thường gặp nhất là supervised learning. Bài Titanic đang làm cũng thuộc dạng này.

Core của supervised learning là toán học. Data được biểu diễn dưới dạng số học, và model là 1 tập hợp các công thức toán học để biến đổi từ input -> output và từ sai số của output so với dữ liệu thực (labels) để điều chỉnh lại các trọng số được sử dụng trong các công thức trong model.

Ví dụ: bộ data có thể biểu diễn trên 1 đồ thị với `x` là feature và `y` là label. 

![data](imgs/data.png)

ML sẽ tìm 1 hàm số `f` để map feature `x` với label `y`, với sai số nhỏ nhất có thể

![learning](imgs/learning.png)

Khi mới khởi tạo, model có thể sẽ đoán ra hàm số như đường màu <font color='red'>đỏ</font>. Trong quá trình training, model có thể sẽ tìm ra hàm số màu <font color='blue'>xanh biển</font> rồi tìm đến giá trị tối ưu màu <font color='green'>xanh lá</font>.

## Convert data về dạng số

Các thư viện ML phổ biến hiện nay đã giúp đẩy phần toán xuống phía dưới và đưa ra những method để training 1 ML model trở nên rất đơn giản cho bất kỳ lập trình viên nào. Tuy nhiên lập trình viên/data scientist vẫn có nhiệm vụ là đưa data về dạng tính toán được.

Xem lại dạng data chúng ta đang có:

In [ ]:
data.info()

Có 4 feature đang ở dạng `category`, có thể convert thành số rất đơn giản:

In [ ]:
pd.get_dummies(data)

In [ ]:
category_columns = data.select_dtypes(['category']).columns
data[category_columns] = data[category_columns].apply(lambda x: x.cat.codes)
data.info()

### Quiz

Tại sao không biến đổi trực tiếp từ data ban đầu thành dạng số mà lại đưa về thành `category` trước?

In [ ]:
%load solutions/quiz_answer.py